In [1]:
import pandas as pd
import os

In [2]:
SCRIPT_DIR_PATH = os.getcwd()
OUTPUT_POSTPROCESSING_DIR_PATH = os.path.dirname(SCRIPT_DIR_PATH)
SSP_MODELING_DIR_PATH = os.path.dirname(OUTPUT_POSTPROCESSING_DIR_PATH)
SSP_RUN_OUTPUT_DIR_PATH = os.path.join(SSP_MODELING_DIR_PATH, "ssp_run_output")
RUN_OUTPUT_DIR_PATH = os.path.join(SSP_RUN_OUTPUT_DIR_PATH, "sisepuede_results_sisepuede_run_2025-09-30T16;13;13.113881")
TABLEAU_DATA_DIR_PATH = os.path.join(SSP_MODELING_DIR_PATH, "tableau", "data")


In [3]:
# --- Load tables ---
ssp_table = pd.read_csv(os.path.join(RUN_OUTPUT_DIR_PATH, "levers_implementation_morocco.csv"))
ssp_table.head()

,sector,subsector,transformer_code,transformer_name,transformer_description,transformer_unit,start_period,maximum_magnitude,strategy_PFLO_LEP,year
0,AFOLU,AGRC,TFR:AGRC:DEC_CH4_RICE,AGRC: Improve rice management,Reduce :math:`\text{CH}_4` emissions from rice...,Fractional reduction in methane emitted from r...,12,0.0,0.0,2015
1,AFOLU,AGRC,TFR:AGRC:DEC_CH4_RICE,AGRC: Improve rice management,Reduce :math:`\text{CH}_4` emissions from rice...,Fractional reduction in methane emitted from r...,12,0.0,0.0,2016
2,AFOLU,AGRC,TFR:AGRC:DEC_CH4_RICE,AGRC: Improve rice management,Reduce :math:`\text{CH}_4` emissions from rice...,Fractional reduction in methane emitted from r...,12,0.0,0.0,2017
3,AFOLU,AGRC,TFR:AGRC:DEC_CH4_RICE,AGRC: Improve rice management,Reduce :math:`\text{CH}_4` emissions from rice...,Fractional reduction in methane emitted from r...,12,0.0,0.0,2018
4,AFOLU,AGRC,TFR:AGRC:DEC_CH4_RICE,AGRC: Improve rice management,Reduce :math:`\text{CH}_4` emissions from rice...,Fractional reduction in methane emitted from r...,12,0.0,0.0,2019


In [4]:
# Create transformation_code by removing "TFR:" from transformer_code
ssp_table["transformation_code"] = ssp_table["transformer_code"].str.replace(r"^TFR:", "", regex=True)

In [5]:
# Descriptions
desp = pd.read_csv("ssp_descriptions.csv")
desp.head()

,transformation_code,transformation_description_long
0,AGRC:DEC_CH4_RICE,Many practices can reduce emissions associated...
1,AGRC:DEC_EXPORTS,Decrease agricultural exports by some percenta...
2,AGRC:DEC_LOSSES_SUPPLY_CHAIN,Reduce waste food waste in the agricultural (c...
3,AGRC:INC_CONSERVATION_AGRICULTURE,Conservation agriculture is the term given to ...
4,AGRC:INC_PRODUCTIVITY,Apply a fractional increase to crop yield fact...


In [6]:
# Stakeholder codes
scodes = pd.read_csv("stakeholder_codes.csv")
scodes["transformation_code"] = scodes["transformation_code"].str.replace(r"^TX:", "", regex=True)
scodes.head()

,transformation_code,transformation_name_stakeholder,sector,subsector,Sector (output),Subsector (output),Example government policies
0,AGRC:DEC_CH4_RICE,Improve rice management,AFOLU,AGRC,AFOLU,Agriculture,NaN
1,AGRC:DEC_EXPORTS,Decrease Exports,AFOLU,AGRC,AFOLU,Agriculture,NaN
2,AGRC:DEC_LOSSES_SUPPLY_CHAIN,Reduce supply chain losses,AFOLU,AGRC,AFOLU,Agriculture,NaN
3,AGRC:INC_CONSERVATION_AGRICULTURE,Expand conservation agriculture,AFOLU,AGRC,AFOLU,Agriculture,NaN
4,AGRC:INC_PRODUCTIVITY,Improve crop productivity,AFOLU,AGRC,AFOLU,Agriculture,NaN


In [7]:
print("ssp_table shape:", ssp_table.shape)
print("desp shape:", desp.shape)

# --- Merge with descriptions ---
ssp_table = ssp_table.merge(desp, on="transformation_code", how="inner")
print("after desp merge:", ssp_table.shape)

ssp_table shape: (2016, 11)
desp shape: (64, 2)
after desp merge: (1980, 12)


In [8]:
scodes.columns

Index(['transformation_code', 'transformation_name_stakeholder', 'sector',
       'subsector', 'Sector (output)', 'Subsector (output)',
       'Example government policies'],
      dtype='object')

In [9]:
# --- Merge with selected stakeholder columns ---
stakeholder_cols = [
    "transformation_code",
    "transformation_name_stakeholder",
    "Sector (output)",
    "Subsector (output)",
    "Example government policies",
]
ssp_table = ssp_table.merge(scodes[stakeholder_cols], on="transformation_code", how="inner")
print("after scodes merge:", ssp_table.shape)

after scodes merge: (1980, 16)


In [10]:
# --- Write result ---
out_path = os.path.join(TABLEAU_DATA_DIR_PATH, "tableau_levers_table_complete.csv")
os.makedirs(os.path.dirname(out_path), exist_ok=True)
ssp_table.to_csv(out_path, index=False)
print(f"Wrote: {out_path}")

Wrote: /Users/tony/Documents/sisepuede_modeling/ssp_morocco/ssp_modeling/tableau/data/tableau_levers_table_complete.csv
